# 출력 파서 (`OutputParser`)

- LLM의 출력을 더 유용/구조화된 형태로 변환함
- 구조화: LLM의 자유 형식 출력을 구조화된 데이터로 변환
- 일관성: 항상 일관된 출력형식 -> 후속 처리에 용이
- 유연성: 다양한 출력 형식(JSON, list, dict)으로 변환 가능

## `PydanticOutputParser`

In [ ]:
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field

load_dotenv()

llm = ChatOpenAI(model='gpt-4.1-nano')

In [ ]:
email_conversation = """From: 김철수 (chulsoo.kim@bikecorporation.me)
To: 이은채 (eunchae@teddyinternational.me)
Subject: "ZENESIS" 자전거 유통 협력 및 미팅 일정 제안

안녕하세요, 이은채 대리님,

저는 바이크코퍼레이션의 김철수 상무입니다. 최근 보도자료를 통해 귀사의 신규 자전거 "ZENESIS"에 대해 알게 되었습니다. 바이크코퍼레이션은 자전거 제조 및 유통 분야에서 혁신과 품질을 선도하는 기업으로, 이 분야에서의 장기적인 경험과 전문성을 가지고 있습니다.

ZENESIS 모델에 대한 상세한 브로슈어를 요청드립니다. 특히 기술 사양, 배터리 성능, 그리고 디자인 측면에 대한 정보가 필요합니다. 이를 통해 저희가 제안할 유통 전략과 마케팅 계획을 보다 구체화할 수 있을 것입니다.

또한, 협력 가능성을 더 깊이 논의하기 위해 다음 주 화요일(1월 15일) 오전 10시에 미팅을 제안합니다. 귀사 사무실에서 만나 이야기를 나눌 수 있을까요?

감사합니다.

김철수
상무이사
바이크코퍼레이션
"""

In [ ]:
from langchain_core.prompts import PromptTemplate

prompt = PromptTemplate.from_template(
    '아래 이메일 내용중 중요한 것만 추출해.\n\n---\n\n{email_conversation}'
)

# print(prompt.format(email_conversation=email_conversation))
chain = prompt | llm 

print('---출력파서 없는 요약---')
print(chain.invoke({'email_conversation': email_conversation}).content)

In [ ]:
class EmailSummary(BaseModel):
    person: str = Field(description='메일 보낸 사람')
    email: str = Field(description='보낸사람의 메일주소')
    subject: str = Field(description='메일 제목')
    summary: str = Field(description='메일 본문 요약')
    date: str = Field(description='메일에 언급된 미팅 날짜와 시간')

parser = PydanticOutputParser(pydantic_object=EmailSummary)
print(parser.get_format_instructions())

In [22]:
prompt = PromptTemplate.from_template(
    """
너는 요약의 신 어시스턴트야. 아래 질문에 맞게 답변을 한국어로 만들어줘

질문: {question}

이메일 내용: {email_conversation}

형식: {format}

"""
)

# 프롬프트 변수들중 일부만 채우기
prompt = prompt.partial(format=parser.get_format_instructions())

In [24]:
chain = prompt | llm | parser

res = chain.invoke(
    {
        'question': '이메일 내용중 중요한 내용을 추출해 줘!',
        'email_conversation': email_conversation
    }
)

res

EmailSummary(person='김철수 상무', email='chulsoo.kim@bikecorporation.me', subject='"ZENESIS" 자전거 유통 협력 및 미팅 일정 제안', summary="바이크코퍼레이션의 김철수 상무가 귀사의 'ZENESIS' 자전거에 대해 상세 브로슈어 요청과 함께 협력 가능성 논의를 위해 1월 15일 오전 10시에 미팅을 제안하고 있습니다.", date='2023년 1월 15일 오전 10시')

In [33]:
from pprint import pprint
pprint(res.model_dump())

{'date': '2023년 1월 15일 오전 10시',
 'email': 'chulsoo.kim@bikecorporation.me',
 'person': '김철수 상무',
 'subject': '"ZENESIS" 자전거 유통 협력 및 미팅 일정 제안',
 'summary': "바이크코퍼레이션의 김철수 상무가 귀사의 'ZENESIS' 자전거에 대해 상세 브로슈어 요청과 함께 협력 가능성 논의를 "
            '위해 1월 15일 오전 10시에 미팅을 제안하고 있습니다.'}
